In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2  
import pyterrier as pt
import os
import numpy as np
import gc

print("Done")

Done


In [2]:
# --- PyTerrier Initialisierung ---
if not pt.started():
    pt.java.init()
print("Done")

/tmp/ipykernel_533361/1499247338.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


Done


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


In [3]:
# --- Datenbankverbindung ---
DATABASE = "longeval-web"
USER = "dis18"
HOST = "db"
PORT = "5432"
PASSWORD = os.getenv("DB_PASSWORD", "dis182425")  

try:
    engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")
    with engine.begin() as connection:
        print("Datenbankverbindung erfolgreich.")
except Exception as e:
    print(f"Fehler bei DB-Verbindung: {e}")
    exit()
print("Done")

Datenbankverbindung erfolgreich.
Done


In [4]:
DATABASE = "longeval-web"
USER = "dis18"
HOST = "db"
PORT = "5432"
PASSWORD = "dis182425"

engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

In [5]:
# --- SQL Queries ---
query_topic = """
SELECT 
    "Topic"."queryid" AS quer_id,    
    "Topic"."text_fr" AS query,
    "sub_collection" 
FROM "Topic"
WHERE "sub_collection" = '2023-03';
"""

query_qrel = """
SELECT DISTINCT
    "Qrel"."relevance" AS relevance,
    "queryid" as quer_id,
    "docid" as doc_id
    
from "Qrel"
WHERE "sub_collection" = '2023-03';


"""

query_document = """
SELECT 
    "Document"."url" AS url,
    "docid" as doc_id
from "Document"
WHERE "sub_collection" IN ('2023-03');
"""

query_vorkommen = """
WITH doc_counts AS (
    SELECT
        "docid",
        COUNT(*) AS "Anzahl_Vorkommen"
    FROM "Document"
    WHERE "sub_collection" IN (
        '2022-06', '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
        '2023-01', '2023-02', '2023-03'
    )
    GROUP BY "docid"
)
SELECT
    "docid",
    "Anzahl_Vorkommen"
FROM doc_counts;
"""


df_topic = pd.read_sql(query_topic, con=engine)
df_qrel = pd.read_sql(query_qrel, con=engine)
df_document = pd.read_sql(query_document, con=engine)
df_vorkommen = pd.read_sql(query_vorkommen, con=engine)

print(df_topic)
print(df_qrel)
print(df_document)
print(df_vorkommen)
print("Done")
# Anzahl eindeutiger Queries in den Qrels
num_qrels = df_qrel['quer_id'].nunique()
print(f"Einzigartige quer_id in df_qrel: {num_qrels}")



     quer_id                                         query sub_collection
0          3                          1ere guerre mondiale        2023-03
1         12                                  abri voiture        2023-03
2         19                        activ crea pole emploi        2023-03
3         24                                 adidas terrex        2023-03
4         28                             aeroport bordeaux        2023-03
...      ...                                           ...            ...
5680   75327                               jean luc bideau        2023-03
5681   75328                             loi censi bouvard        2023-03
5682   75352                              radiologie blois        2023-03
5683   75382  archives departementales de loire atlantique        2023-03
5684   75398                            gateau paris brest        2023-03

[5685 rows x 3 columns]
Empty DataFrame
Columns: [relevance, quer_id, doc_id]
Index: []
                       

In [6]:
# --- BM25 Daten laden ---

bm25_file = "datasets/LongEval-Web/runs/longeval-web-fr-2023-03-BM25.gz"
bm_df = pt.io.read_results(bm25_file)
print(bm_df)
num_bm = bm_df['qid'].nunique()
print(f"Einzigartige quer_id in bm_df: {num_bm}")

print('Done')




           qid    docno  rank      score       name
0            3  2214755     0  24.226631  pyterrier
1            3   684186     1  23.345397  pyterrier
2            3   637997     2  23.149936  pyterrier
3            3   430968     3  22.982027  pyterrier
4            3   160081     4  22.781866  pyterrier
...        ...      ...   ...        ...        ...
5468763  75398   235782   995  15.418909  pyterrier
5468764  75398  2681929   996  15.411665  pyterrier
5468765  75398  3128573   997  15.409610  pyterrier
5468766  75398    57032   998  15.408249  pyterrier
5468767  75398  2322020   999  15.407915  pyterrier

[5468768 rows x 5 columns]
Einzigartige quer_id in bm_df: 5628
Done


In [7]:
# --- Hilfsfunktion zur Speicheroptimierung bm_df ---
def optimize_df_types(df: pd.DataFrame) -> pd.DataFrame:
    """Optimizes DataFrame types in-place for memory."""
    for col in df.columns:
        col_type = df[col].dtype
        try:
            if col_type == 'float64':
                df[col] = df[col].astype(np.float32)
            elif col_type == 'int64':
                df[col] = pd.to_numeric(df[col], downcast='integer')
            elif col_type == 'object' and df[col].nunique() / len(df) < 0.5:
                 df[col] = df[col].astype('category')
        except Exception as e:

             pass 
    return df

list_of_dfs = [bm_df]
optimized_list = [optimize_df_types(df) for df in list_of_dfs]
print("Done")

Done


In [8]:
# --- Hilfsfunktion zur Speicheroptimierung ---
def optimize_df_types(df: pd.DataFrame) -> pd.DataFrame:
    """Optimizes DataFrame types in-place for memory."""
    for col in df.columns:
        col_type = df[col].dtype
        try:
            if col_type == 'float64':
                df[col] = df[col].astype(np.float32)
            elif col_type == 'int64':
                df[col] = pd.to_numeric(df[col], downcast='integer')
            elif col_type == 'object' and df[col].nunique() / len(df) < 0.5:
                 df[col] = df[col].astype('category')
        except Exception as e:

             pass 
    return df

list_of_dfs = [df_topic, df_qrel, df_document, df_vorkommen]
optimized_list = [optimize_df_types(df) for df in list_of_dfs]

print("Done")

Done


In [9]:
df_vorkommen['docid'] = df_vorkommen['docid'].str.replace(r'^doc', '', regex=True)

print(df_vorkommen)


          docid  Anzahl_Vorkommen
0             1                 1
1            10                10
2           100                 4
3          1000                 9
4         10000                 2
...         ...               ...
3453823  999995                 2
3453824  999996                 6
3453825  999997                 6
3453826  999998                 4
3453827  999999                 3

[3453828 rows x 2 columns]


In [10]:

bm_df['qid'] = bm_df['qid'].astype(str)
df_topic['quer_id'] = df_topic['quer_id'].astype(str)

print("Einzigartige qid in bm_df:", bm_df['qid'].nunique())


Einzigartige qid in bm_df: 5628


In [11]:
# Anzahl eindeutiger Queries in den Qrels
num_qrels = df_qrel['quer_id'].nunique()
print(f"Einzigartige quer_id in df_qrel: {num_qrels}")

# Anzahl eindeutiger Queries im BM25-Run
num_bm = bm_df['qid'].nunique()
print(f"Einzigartige qid in bm_df: {num_bm}")

Einzigartige quer_id in df_qrel: 0
Einzigartige qid in bm_df: 5628


In [12]:
Time_Tags = pd.read_csv('Expanded/Topics_With_TimeTags.csv')

Time_Tags_202301 = Time_Tags[Time_Tags['sub_collection'] == '2023-03'].copy()
print(Time_Tags_202301)
print(bm_df)
unique_queryids = Time_Tags_202301['queryid'].nunique()

print(f"Anzahl einzigartiger queryid in sub_collection 2023-01: {unique_queryids}")

                                         id  queryid  text_en  \
90475  e9bb512e-a0c6-4404-9a47-bdc59ad62854        3      NaN   
90476  7e316f55-7f1a-4d74-89d6-b79c5eb00941       12      NaN   
90477  4509614f-3bca-4d1a-9a9a-823a3be07a78       19      NaN   
90478  b493fb56-98bb-43c7-ace8-a47a92a782ad       24      NaN   
90479  535cbdd8-bca5-4e46-ad91-afa76e0361cc       28      NaN   
...                                     ...      ...      ...   
96156  f0ae4175-7671-4476-8a44-ee03c11e6469    75327      NaN   
96157  6b112375-2d60-4cd6-92c9-444388982342    75328      NaN   
96158  3f099f52-10ec-4f40-81bb-94656e17e1e4    75352      NaN   
96159  e7894d83-8380-42c9-9d4c-42e1a005c2e9    75382      NaN   
96160  4d250be6-6211-4243-90ec-b9efd38c5ecd    75398      NaN   

                                              query sub_collection split  tag  
90475                          1ere guerre mondiale        2023-03  test  0.2  
90476                                  abri voiture        

In [13]:
bm_df['qid'] = bm_df['qid'].astype(str)
Time_Tags_202301['queryid'] = Time_Tags_202301['queryid'].astype(str)

Final_df = pd.merge(
    bm_df,
    Time_Tags_202301,
    left_on="qid",
    right_on="queryid",
    how="inner"
)
print(Final_df)

print(Final_df.sort_values(by='score', ascending=False))


           qid    docno  rank      score       name  \
0            3  2214755     0  24.226631  pyterrier   
1            3   684186     1  23.345398  pyterrier   
2            3   637997     2  23.149937  pyterrier   
3            3   430968     3  22.982027  pyterrier   
4            3   160081     4  22.781866  pyterrier   
...        ...      ...   ...        ...        ...   
5468763  75398   235782   995  15.418909  pyterrier   
5468764  75398  2681929   996  15.411665  pyterrier   
5468765  75398  3128573   997  15.409611  pyterrier   
5468766  75398    57032   998  15.408249  pyterrier   
5468767  75398  2322020   999  15.407914  pyterrier   

                                           id queryid  text_en  \
0        e9bb512e-a0c6-4404-9a47-bdc59ad62854       3      NaN   
1        e9bb512e-a0c6-4404-9a47-bdc59ad62854       3      NaN   
2        e9bb512e-a0c6-4404-9a47-bdc59ad62854       3      NaN   
3        e9bb512e-a0c6-4404-9a47-bdc59ad62854       3      NaN   
4        

In [14]:
import pandas as pd


df_vorkommen_renamed = df_vorkommen.rename(columns={'docid': 'docno'})


Final_df = pd.merge(
    Final_df,
    df_vorkommen_renamed,
    on='docno',
    how='left'   
)

print(Final_df.sort_values(by='Anzahl_Vorkommen', ascending=True))



           qid    docno  rank      score       name  \
1165851   3908  3442573   344  11.438331  pyterrier   
4119332  55971  3440702   336  16.703697  pyterrier   
4119336  55971  3438635   340  16.665726  pyterrier   
4119274  55971  3452230   278  17.273876  pyterrier   
4119468  55971  3453154   472  15.502724  pyterrier   
...        ...      ...   ...        ...        ...   
5456203  75298  3451978   435  19.782284  pyterrier   
5457998  75303  3452053   230  29.072803  pyterrier   
5459873  75306  3452017   105  13.361832  pyterrier   
5463774  75327  3452071     6  28.095406  pyterrier   
5466382  75352  3451437   614  15.595189  pyterrier   

                                           id queryid  text_en  \
1165851  14b91b00-eb8d-477a-967a-d72fadda26e8    3908      NaN   
4119332  87a79c4e-0348-4499-bbb2-5de6080f875a   55971      NaN   
4119336  87a79c4e-0348-4499-bbb2-5de6080f875a   55971      NaN   
4119274  87a79c4e-0348-4499-bbb2-5de6080f875a   55971      NaN   
4119468  

In [15]:
Final_df['Anzahl_Vorkommen'] = Final_df['Anzahl_Vorkommen'].fillna(1.0)

print(Final_df['Anzahl_Vorkommen'].isnull().sum(), "NaNs nach fillna")
print(Final_df.sort_values(by='Anzahl_Vorkommen', ascending=True).head())


0 NaNs nach fillna
           qid    docno  rank      score       name  \
3143358  44263  3449924   330  13.768674  pyterrier   
3143301  44263  3440919   273  14.594456  pyterrier   
3143304  44263  3453225   276  14.577655  pyterrier   
1167806   3923  3444315   299  15.633690  pyterrier   
1167692   3923  3447916   185  16.247210  pyterrier   

                                           id queryid  text_en  \
3143358  cb728515-bd84-4e30-8123-3e0374565c57   44263      NaN   
3143301  cb728515-bd84-4e30-8123-3e0374565c57   44263      NaN   
3143304  cb728515-bd84-4e30-8123-3e0374565c57   44263      NaN   
1167806  b781e44e-2864-4dd7-8c7d-4702fd655fab    3923      NaN   
1167692  b781e44e-2864-4dd7-8c7d-4702fd655fab    3923      NaN   

                    query sub_collection split  tag  Anzahl_Vorkommen  
3143358    enquete emploi        2023-03  test  0.8               1.0  
3143301    enquete emploi        2023-03  test  0.8               1.0  
3143304    enquete emploi        2023

In [16]:
print(Final_df.dtypes)


Final_df.info()


qid                   object
docno                 object
rank                   int16
score                float32
name                category
id                    object
queryid               object
text_en              float64
query                 object
sub_collection        object
split                 object
tag                  float64
Anzahl_Vorkommen     float64
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5468768 entries, 0 to 5468767
Data columns (total 13 columns):
 #   Column            Dtype   
---  ------            -----   
 0   qid               object  
 1   docno             object  
 2   rank              int16   
 3   score             float32 
 4   name              category
 5   id                object  
 6   queryid           object  
 7   text_en           float64 
 8   query             object  
 9   sub_collection    object  
 10  split             object  
 11  tag               float64 
 12  Anzahl_Vorkommen  float64 
dtypes: category

In [17]:
Final_df['Anzahl_Vorkommen'] = Final_df['Anzahl_Vorkommen'].clip(upper=15)

Final_df['recency'] = 1 / (1 + np.log1p(Final_df['Anzahl_Vorkommen']))

lam = 1.0
Final_df['rerank_score'] = Final_df['score'] + lam * Final_df['tag'] * Final_df['recency']

Final_df = Final_df.drop(columns=['score', 'recency'])

Final_df = Final_df.sort_values(['qid','rerank_score'], ascending=[True,False])

print(Final_df.head())




         qid    docno  rank       name                                    id  \
315404  1000    12254     0  pyterrier  dad5fc7e-50db-4e96-bd0d-597e224c87ca   
315405  1000  1641568     1  pyterrier  dad5fc7e-50db-4e96-bd0d-597e224c87ca   
315406  1000  3376776     2  pyterrier  dad5fc7e-50db-4e96-bd0d-597e224c87ca   
315407  1000  3391193     3  pyterrier  dad5fc7e-50db-4e96-bd0d-597e224c87ca   
315408  1000  3384961     4  pyterrier  dad5fc7e-50db-4e96-bd0d-597e224c87ca   

       queryid  text_en                     query sub_collection split  tag  \
315404    1000      NaN  ent mon bureau numerique        2023-03  test  0.2   
315405    1000      NaN  ent mon bureau numerique        2023-03  test  0.2   
315406    1000      NaN  ent mon bureau numerique        2023-03  test  0.2   
315407    1000      NaN  ent mon bureau numerique        2023-03  test  0.2   
315408    1000      NaN  ent mon bureau numerique        2023-03  test  0.2   

        Anzahl_Vorkommen  rerank_score  
315

In [18]:
Final_df_for_evaluation = (
    Final_df[['qid', 'docno', 'rerank_score']]
    .rename(columns={'rerank_score': 'score'})
    .copy()
)
Final_df_for_evaluation['qid']   = Final_df_for_evaluation['qid'].astype(str)
Final_df_for_evaluation['docno'] = Final_df_for_evaluation['docno'].astype(str)
Final_df_for_evaluation['score'] = Final_df_for_evaluation['score'].astype(float)


df_qrel = df_qrel.rename(columns={'doc_id': 'docno', 'quer_id': 'qid'})
df_qrel['docno'] = df_qrel['docno'].astype(str).str.replace('doc', '', regex=False)
df_qrel['qid']   = df_qrel['qid'].astype(str)

print("Head of Final_df_for_evaluation:")
print(Final_df_for_evaluation)

print("\nHead of df_qrel:")
print(df_qrel)


Head of Final_df_for_evaluation:
         qid    docno      score
315404  1000    12254  40.232505
315405  1000  1641568  40.085300
315406  1000  3376776  38.832423
315407  1000  3391193  38.286070
315408  1000  3384961  38.166529
...      ...      ...        ...
315398   998  2675987  15.701060
315400   998  1293841  15.684775
315403   998  1908489  15.683617
315401   998  3055126  15.681539
315402   998   838659  15.677192

[5468768 rows x 3 columns]

Head of df_qrel:
Empty DataFrame
Columns: [relevance, qid, docno]
Index: []


In [19]:
df_qrel = df_qrel.rename(columns={
    'quer_id':   'qid',
    'doc_id':    'docno',
    'relevance': 'label'
})

df_qrel['label'] = df_qrel['label'].astype(float).astype(int)

df_qrel['qid']   = df_qrel['qid'].astype(str)
df_qrel['docno'] = df_qrel['docno'].astype(str)

Final_df_for_evaluation = Final_df[['qid', 'docno', 'rerank_score']].rename(
    columns={'rerank_score': 'score'}
).copy()
Final_df_for_evaluation['qid']   = Final_df_for_evaluation['qid'].astype(str)
Final_df_for_evaluation['docno'] = Final_df_for_evaluation['docno'].astype(str)
Final_df_for_evaluation['score'] = Final_df_for_evaluation['score'].astype(float)

results = pt.Evaluate(
    res   = Final_df_for_evaluation,
    qrels = df_qrel,
    metrics = ["map", "ndcg_cut_10", "recall_100"],
  
 
)

print("Evaluationsergebnisse (Durchschnitt über alle Queries):")
for metric, value in results.items():
    print(f"{metric}: {value:.4f}")



Evaluationsergebnisse (Durchschnitt über alle Queries):
map: nan
ndcg_cut_10: nan
recall_100: nan


In [20]:
run_name = 'tag'

Final_df['rank'] = (
    Final_df
      .sort_values(['qid', 'rerank_score'], ascending=[True, False])
      .groupby('qid')
      .cumcount() + 1
)

run_df = Final_df[['qid', 'docno', 'rank', 'rerank_score']].copy()
run_df['Q0']       = 'Q0'
run_df['run_name'] = run_name

run_df['docno'] = 'doc-' + run_df['docno'].astype(str)

run_df = run_df[['qid', 'Q0', 'docno', 'rank', 'rerank_score', 'run_name']]

max_rank = 1000
run_df = run_df[run_df['rank'] <= max_rank]

run_df.to_csv('trec_run_2023-03_final.run', sep=' ', header=False, index=False)

print(f"Run-Datei geschrieben als trec_run_2023-08_2.run mit Top {max_rank}-Ergebnissen je Query")

Run-Datei geschrieben als trec_run_2023-08_2.run mit Top 1000-Ergebnissen je Query
